In [3]:
import pandas as pd 
from datetime import datetime, timedelta
from sqlalchemy import create_engine, text

In [4]:
# Create an engine that stores data in the local directory's
engine = create_engine('sqlite:///app.db', echo=True)

In [13]:
create_event_campaigns_table = text('''CREATE TABLE IF NOT EXISTS event_campaigns (
    event_id INTEGER PRIMARY KEY AUTOINCREMENT,
    event_date DATETIME DEFAULT CURRENT_TIMESTAMP,
    campaign_id INTEGER NOT NULL,
    company_id INTEGER NOT NULL,                       
    title VARCHAR(255),
    description TEXT,
    start_date DATE,
    end_date DATE,
    campaign_status VARCHAR(100),
    campaign_code VARCHAR(100),
    target_audience VARCHAR(255),
    budget DECIMAL(10, 2),
    actual_spending DECIMAL(10, 2),
    campaign_result TEXT,
    FOREIGN KEY (campaign_id) REFERENCES campaigns(campaign_id) ON DELETE CASCADE
);''')

In [16]:
with engine.connect() as connection:
    connection.execute(text('DROP TABLE IF EXISTS event_campaigns'))

2024-04-18 09:43:36,703 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 09:43:36,703 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS event_campaigns
2024-04-18 09:43:36,705 INFO sqlalchemy.engine.Engine [generated in 0.00229s] ()
2024-04-18 09:43:36,713 INFO sqlalchemy.engine.Engine ROLLBACK


In [17]:
# Assuming 'engine' is already defined and imported from SQLAlchemy setup
with engine.connect() as connection:
    connection.execute(create_event_campaigns_table)
    print("table created successfully.")

2024-04-18 09:43:41,322 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 09:43:41,323 INFO sqlalchemy.engine.Engine CREATE TABLE IF NOT EXISTS event_campaigns (
    event_id INTEGER PRIMARY KEY AUTOINCREMENT,
    event_date DATETIME DEFAULT CURRENT_TIMESTAMP,
    campaign_id INTEGER NOT NULL,
    company_id INTEGER NOT NULL,                       
    title VARCHAR(255),
    description TEXT,
    start_date DATE,
    end_date DATE,
    campaign_status VARCHAR(100),
    campaign_code VARCHAR(100),
    target_audience VARCHAR(255),
    budget DECIMAL(10, 2),
    actual_spending DECIMAL(10, 2),
    campaign_result TEXT,
    FOREIGN KEY (campaign_id) REFERENCES campaigns(campaign_id) ON DELETE CASCADE
);
2024-04-18 09:43:41,323 INFO sqlalchemy.engine.Engine [generated in 0.00165s] ()
table created successfully.
2024-04-18 09:43:41,331 INFO sqlalchemy.engine.Engine ROLLBACK


In [18]:
pd.read_sql('SELECT * FROM event_campaigns',engine)

2024-04-18 09:43:45,962 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 09:43:45,962 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM event_campaigns")
2024-04-18 09:43:45,963 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 09:43:45,964 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM event_campaigns")
2024-04-18 09:43:45,964 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 09:43:45,966 INFO sqlalchemy.engine.Engine SELECT * FROM event_campaigns
2024-04-18 09:43:45,966 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 09:43:45,970 INFO sqlalchemy.engine.Engine ROLLBACK


,event_id,event_date,campaign_id,company_id,title,description,start_date,end_date,campaign_status,campaign_code,target_audience,budget,actual_spending,campaign_result


In [2]:
def get_event_campaign(**kwargs):
    if not kwargs:
        print("No parameters provided to search for event_campaigns.")
        return None
    
    # Constructing the WHERE clause based on kwargs
    conditions = [f"{key} = :{key}" for key in kwargs]
    where_clause = " AND ".join(conditions)
    
    select_query = text(f"SELECT * FROM event_campaigns WHERE {where_clause}")

    with engine.connect() as connection:
        result = connection.execute(select_query, kwargs).fetchone()
        if result:
            event_campaign_dict = {
                'event_id': result[0],
                'event_date': result[1],
                'campaign_id': result[2],
                'company_id': result[3],
                'title': result[4],
                'description': result[5],
                'start_date': result[6],
                'end_date': result[7],
                'campaign_status': result[8],
                'campaign_code':result[9],
                'target_audience': result[10],
                'budget':result[11],
                'actual_spending':result[12],
                'campaign_result':result[13]
            }
            return event_campaign_dict
        else:
            print("Company not found.")
            return None
        
def add_event_campaign(campaign_id,company_id,**kwargs):
    data = {'campaign_id': campaign_id, 'company_id': company_id, **kwargs}
    columns = ', '.join(data.keys())
    placeholders = ', '.join(f":{key}" for key in data.keys())

    insert_query = text(f"""
        INSERT INTO event_campaigns ({columns})
        VALUES ({placeholders});
    """)
    with engine.begin() as connection:
        connection.execute(insert_query, data)
        print(f"Event added: Campaign ID {campaign_id}, Creator ID {company_id}, Details: {kwargs}")

In [1]:
def modify_event(event_id, **kwargs):
    updates = ', '.join(f"{key} = :{key}" for key in kwargs)
    update_query = text(f"""
        UPDATE event_campaigns
        SET {updates}
        WHERE event_id = :event_id;
    """)
    kwargs['event_id'] = event_id  # Include for WHERE clause
    with engine.begin() as connection:
        result = connection.execute(update_query, kwargs)
        if result.rowcount > 0:
            print(f"Event ID {event_id} updated successfully.")
        else:
            print(f"No event found with ID {event_id}.")

In [19]:
add_event_campaign(1,2)

2024-04-18 09:43:58,782 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 09:43:58,783 INFO sqlalchemy.engine.Engine 
        INSERT INTO event_campaigns (campaign_id, company_id)
        VALUES (?, ?);
    
2024-04-18 09:43:58,783 INFO sqlalchemy.engine.Engine [cached since 257.1s ago] (1, 2)
Event added: Campaign ID 1, Creator ID 2, Details: {}
2024-04-18 09:43:58,807 INFO sqlalchemy.engine.Engine COMMIT


In [21]:
pd.read_sql('SELECT * FROM event_campaigns',engine)

2024-04-18 09:44:17,328 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 09:44:17,329 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM event_campaigns")
2024-04-18 09:44:17,330 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 09:44:17,331 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM event_campaigns")
2024-04-18 09:44:17,331 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 09:44:17,332 INFO sqlalchemy.engine.Engine SELECT * FROM event_campaigns
2024-04-18 09:44:17,333 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 09:44:17,335 INFO sqlalchemy.engine.Engine ROLLBACK


,event_id,event_date,campaign_id,company_id,title,description,start_date,end_date,campaign_status,campaign_code,target_audience,budget,actual_spending,campaign_result
0,1,2024-04-18 07:43:58,1,2,None,None,None,None,None,None,None,None,None,None
